In [1]:
import pandas as pd

vessels_df = pd.read_csv('../data/original_data/vessels.csv', sep='|')

train_df = pd.read_csv("../data/processed_data/train.csv")

test_df = pd.read_csv("../data/processed_data/test.csv")

print(len(train_df))
print(len(test_df))

vessels_df.head()


1522065
51739


,shippingLineId,vesselId,CEU,DWT,GT,NT,vesselType,breadth,depth,draft,enginePower,freshWater,fuel,homePort,length,maxHeight,maxSpeed,maxWidth,rampCapacity,yearBuilt
0,61a8e672f9cba188601e84ab,61e9f38eb937134a3c4bfd8b,6500,21200.0,58684,17606.0,83.0,32.0,22.20,NaN,0.0,NaN,NaN,OSLO,199.00,5.0,18.6,15.2,150.0,2000
1,61ec94f1a8cafc0e93f0e92a,61e9f38eb937134a3c4bfd8d,4902,12325.0,46800,NaN,83.0,31.0,NaN,NaN,14220.0,NaN,NaN,MONROVIA,182.00,NaN,NaN,NaN,NaN,2006
2,61e213d5d612676a0f0fb755,61e9f38eb937134a3c4bfd8f,5000,13059.0,46800,NaN,83.0,31.0,NaN,NaN,14220.0,NaN,NaN,SAINT JOHN'S,182.00,NaN,NaN,NaN,NaN,2010
3,61be24574ea00ae59d0fe388,61e9f38eb937134a3c4bfd91,4200,12588.0,39362,NaN,83.0,28.0,NaN,NaN,11060.0,NaN,NaN,NaN,167.00,NaN,NaN,NaN,NaN,2011
4,61a8e673f9cba188601e84ae,61e9f390b937134a3c4bfd93,7450,21052.0,75528,24391.0,83.0,37.2,22.23,NaN,13140.0,491.47,3236.78,Panama,199.98,NaN,NaN,NaN,NaN,2018


In [2]:
vessels_df = pd.merge(vessels_df, train_df[['vesselId']], on='vesselId', how='inner').drop_duplicates()

print(f"Rows of vessels_df that have a vesselId that is also present in train_df: {len(vessels_df)}")

Rows of vessels_df that have a vesselId that is also present in train_df: 688


In [3]:
vessels_df['enginePower'] = vessels_df['enginePower'].fillna(14121.0) # Median engine power value found in eda
vessels_df.loc[vessels_df['enginePower'] == 0, 'enginePower'] = 14121.0 # Median engine power value found in eda

vessels_df['breadth'] = vessels_df['breadth'].fillna(32) # Median breadth valud found in eda

vessels_df['vesselType'] = vessels_df['vesselType'].fillna(83) # Most typical vesselType valud found in eda

vessels_df['DWT'] = vessels_df['DWT'].fillna(18758) # Median DWT valud found in eda

In [4]:
train_set = vessels_df[['vesselId', 'enginePower', 'CEU', 'GT', 'breadth', 'length', 'vesselType', 'DWT', 'maxSpeed']].reset_index(drop=True)

train_set.head()

,vesselId,enginePower,CEU,GT,breadth,length,vesselType,DWT,maxSpeed
0,61e9f38eb937134a3c4bfd8b,14121.0,6500,58684,32.0,199.00,83.0,21200.0,18.6
1,61e9f38eb937134a3c4bfd8d,14220.0,4902,46800,31.0,182.00,83.0,12325.0,NaN
2,61e9f38eb937134a3c4bfd8f,14220.0,5000,46800,31.0,182.00,83.0,13059.0,NaN
3,61e9f38eb937134a3c4bfd91,11060.0,4200,39362,28.0,167.00,83.0,12588.0,NaN
4,61e9f390b937134a3c4bfd93,13140.0,7450,75528,37.2,199.98,83.0,21052.0,NaN


In [5]:
min_values = train_set.min()
max_values = train_set.max()

# Combine the min and max values into a single DataFrame for easier viewing
min_max_df = pd.DataFrame({'Min': min_values, 'Max': max_values})

# Display the result
print(min_max_df)

                                  Min                        Max
vesselId     61e9f38eb937134a3c4bfd8b  clh6aqawa0007gh0z9h6zi9bo
enginePower                    1500.0                    36000.0
CEU                                 0                       8500
GT                               8659                     100430
breadth                          18.0                       42.0
length                           99.9                      296.0
vesselType                       14.0                       83.0
DWT                            3222.0                   108650.0
maxSpeed                         16.7                       23.3


In [6]:
from sklearn.preprocessing import MinMaxScaler

vessel_ids = train_set['vesselId']
vessel_types = train_set['vesselType']
train_set = train_set.drop(columns=['vesselId', 'vesselType'])


# Initialize the MinMaxScaler
scaler = MinMaxScaler(feature_range=(0, 1))

# Fit and transform the data to normalize it
train_set = pd.DataFrame(scaler.fit_transform(train_set), columns=train_set.columns)

vessel_type_encoded = pd.get_dummies(vessel_types, prefix='vesselType')

train_set = pd.concat([vessel_ids.reset_index(drop=True), vessel_type_encoded.reset_index(drop=True), train_set.reset_index(drop=True)], axis=1)

# Display the normalized DataFrame
print(train_set.head())

                   vesselId  vesselType_14.0  vesselType_21.0  \
0  61e9f38eb937134a3c4bfd8b            False            False   
1  61e9f38eb937134a3c4bfd8d            False            False   
2  61e9f38eb937134a3c4bfd8f            False            False   
3  61e9f38eb937134a3c4bfd91            False            False   
4  61e9f390b937134a3c4bfd93            False            False   

   vesselType_83.0  enginePower       CEU        GT   breadth    length  \
0             True     0.365826  0.764706  0.545107  0.583333  0.505354   
1             True     0.368696  0.576706  0.415611  0.541667  0.418664   
2             True     0.368696  0.588235  0.415611  0.541667  0.418664   
3             True     0.277101  0.494118  0.334561  0.416667  0.342172   
4             True     0.337391  0.876471  0.728651  0.800000  0.510352   

        DWT  maxSpeed  
0  0.170524  0.287879  
1  0.086343       NaN  
2  0.093305       NaN  
3  0.088838       NaN  
4  0.169120       NaN  


In [7]:
nan_counts = train_set.isna().sum()

print(nan_counts)

print(f"Rows: {len(train_set)}")

vesselId             0
vesselType_14.0      0
vesselType_21.0      0
vesselType_83.0      0
enginePower          0
CEU                  0
GT                   0
breadth              0
length               0
DWT                  0
maxSpeed           479
dtype: int64
Rows: 688


In [8]:
print(f"Length of all data: {len(train_set)}")

x_to_pred = train_set[(train_set["maxSpeed"] == 0) | (train_set["maxSpeed"].isna())].drop(columns=["maxSpeed"])
train_set = train_set[(train_set["maxSpeed"] != 0) & (train_set["maxSpeed"].notna())]

vessel_ids_x_to_pred = x_to_pred['vesselId']
vessel_ids_train_set = train_set['vesselId']
x_to_pred = x_to_pred.drop(columns=['vesselId'])
train_set = train_set.drop(columns=['vesselId'])



print(f"Length of x to pred: {len(x_to_pred)}")
print(f"Length of train_set: {len(train_set)}")

train_set.head()


Length of all data: 688
Length of x to pred: 480
Length of train_set: 208


,vesselType_14.0,vesselType_21.0,vesselType_83.0,enginePower,CEU,GT,breadth,length,DWT,maxSpeed
0,False,False,True,0.365826,0.764706,0.545107,0.583333,0.505354,0.170524,0.287879
5,False,False,True,0.339130,0.729412,0.543483,0.594167,0.509944,0.174470,0.803030
6,False,False,True,0.318841,0.592471,0.438788,0.591667,0.408414,0.114467,0.893939
7,False,False,True,0.340870,0.733765,0.553922,0.594167,0.510301,0.148993,0.787879
8,False,False,True,0.340870,0.731176,0.553421,0.594167,0.510301,0.146508,0.818182


In [9]:
from sklearn.model_selection import train_test_split

# Separate features (X) and the label (y)
X = train_set.drop(columns=["maxSpeed"])  # Features
y = train_set["maxSpeed"]  # Label

# Split the data: 80% training, 20% test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Display the shapes of the resulting splits
print(f"X_train shape: {X_train.shape}")
print(f"X_test shape: {X_test.shape}")
print(f"y_train shape: {y_train.shape}")
print(f"y_test shape: {y_test.shape}")

X_train shape: (166, 9)
X_test shape: (42, 9)
y_train shape: (166,)
y_test shape: (42,)


In [10]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import root_mean_squared_error

rf_regressor = RandomForestRegressor(n_estimators=100, random_state=42)
rf_regressor.fit(X_train, y_train)
y_pred = rf_regressor.predict(X_test)
rmse = root_mean_squared_error(y_test, y_pred)
print(f"RMSE: {rmse}")


RMSE: 0.051077565243268085


In [11]:
from sklearn.linear_model import LinearRegression
l_regressor = LinearRegression()
l_regressor.fit(X_train, y_train)
y_pred = l_regressor.predict(X_test)

rmse = root_mean_squared_error(y_test, y_pred)
print(f"RMSE: {rmse}")

RMSE: 0.08741483942636154


In [12]:
from sklearn.ensemble import GradientBoostingRegressor
gb_regressor = GradientBoostingRegressor(random_state=42)
gb_regressor.fit(X_train, y_train)
y_pred = gb_regressor.predict(X_test)

rmse = root_mean_squared_error(y_test, y_pred)
print(f"RMSE: {rmse}")

RMSE: 0.047535987859532586


In [13]:
import xgboost as xgb

xg_regressor = xgb.XGBRegressor(objective='reg:squarederror', n_estimators=100, random_state=42)
xg_regressor.fit(X_train, y_train)
y_pred = xg_regressor.predict(X_test)

rmse = root_mean_squared_error(y_test, y_pred)
print(f"RMSE: {rmse}")

RMSE: 0.05663572998006568


In [14]:
y = gb_regressor.predict(x_to_pred)

x_to_pred["maxSpeed"] = y

train_set['vesselId'] = vessel_ids_train_set
x_to_pred['vesselId'] = vessel_ids_x_to_pred

# Step 3: Combine x_to_pred with the original train_set
complete_vessel_df = pd.concat([train_set, x_to_pred], axis=0)

# Optionally, reset the index for the full dataset
complete_vessel_df = complete_vessel_df.reset_index(drop=True)

# Display the final dataset
print(f"Length of full dataset: {len(complete_vessel_df)}")
complete_vessel_df.head()


Length of full dataset: 688


,vesselType_14.0,vesselType_21.0,vesselType_83.0,enginePower,CEU,GT,breadth,length,DWT,maxSpeed,vesselId
0,False,False,True,0.365826,0.764706,0.545107,0.583333,0.505354,0.170524,0.287879,61e9f38eb937134a3c4bfd8b
1,False,False,True,0.339130,0.729412,0.543483,0.594167,0.509944,0.174470,0.803030,61e9f391b937134a3c4bfd95
2,False,False,True,0.318841,0.592471,0.438788,0.591667,0.408414,0.114467,0.893939,61e9f391b937134a3c4bfd97
3,False,False,True,0.340870,0.733765,0.553922,0.594167,0.510301,0.148993,0.787879,61e9f392b937134a3c4bfd99
4,False,False,True,0.340870,0.731176,0.553421,0.594167,0.510301,0.146508,0.818182,61e9f392b937134a3c4bfd9b


In [15]:
train_df = pd.merge(train_df, complete_vessel_df, on='vesselId', how='left')

train_df.head()

,time,cog,sog,rot,heading,navstat,etaRaw,latitude,longitude,vesselId,...,vesselType_14.0,vesselType_21.0,vesselType_83.0,enginePower,CEU,GT,breadth,length,DWT,maxSpeed
0,0.031663,0.858217,17.1,-6,316,0,01-08 06:00,7.50361,77.58340,61e9f38eb937134a3c4bfd8b,...,False,False,True,0.365826,0.764706,0.545107,0.583333,0.505354,0.170524,0.287879
1,0.031707,0.856825,17.3,5,313,0,01-14 23:30,7.57302,77.49505,61e9f38eb937134a3c4bfd8b,...,False,False,True,0.365826,0.764706,0.545107,0.583333,0.505354,0.170524,0.287879
2,0.031757,0.854596,16.9,5,312,0,01-14 23:30,7.65043,77.39404,61e9f38eb937134a3c4bfd8b,...,False,False,True,0.365826,0.764706,0.545107,0.583333,0.505354,0.170524,0.287879
3,0.031798,0.857660,16.9,6,313,0,01-14 23:30,7.71275,77.31394,61e9f38eb937134a3c4bfd8b,...,False,False,True,0.365826,0.764706,0.545107,0.583333,0.505354,0.170524,0.287879
4,0.031838,0.855153,16.3,7,313,0,01-14 23:30,7.77191,77.23585,61e9f38eb937134a3c4bfd8b,...,False,False,True,0.365826,0.764706,0.545107,0.583333,0.505354,0.170524,0.287879


In [16]:
test_df = pd.merge(test_df, complete_vessel_df, on='vesselId', how='left')

test_df.head()

,ID,vesselId,time,scaling_factor,week_of_the_year,day_of_the_year,avg_lat_change_1_step,avg_lat_change_2_steps,avg_lon_change_1_step,avg_lon_change_2_steps,vesselType_14.0,vesselType_21.0,vesselType_83.0,enginePower,CEU,GT,breadth,length,DWT,maxSpeed
0,4,61e9f38eb937134a3c4bfd8d,0.349750,0.3,0.346154,0.350685,-0.000462,-0.000924,-0.001555,-0.003109,False,False,True,0.368696,0.576706,0.415611,0.541667,0.418664,0.086343,0.837412
1,201,61e9f38eb937134a3c4bfd8d,0.349802,0.3,0.346154,0.350685,-0.000462,-0.000924,-0.001555,-0.003109,False,False,True,0.368696,0.576706,0.415611,0.541667,0.418664,0.086343,0.837412
2,583,61e9f38eb937134a3c4bfd8d,0.349904,0.3,0.346154,0.350685,-0.000462,-0.000924,-0.001555,-0.003109,False,False,True,0.368696,0.576706,0.415611,0.541667,0.418664,0.086343,0.837412
3,701,61e9f38eb937134a3c4bfd8d,0.349938,0.3,0.346154,0.350685,-0.000462,-0.000924,-0.001555,-0.003109,False,False,True,0.368696,0.576706,0.415611,0.541667,0.418664,0.086343,0.837412
4,829,61e9f38eb937134a3c4bfd8d,0.349961,0.3,0.346154,0.350685,-0.000462,-0.000924,-0.001555,-0.003109,False,False,True,0.368696,0.576706,0.415611,0.541667,0.418664,0.086343,0.837412


In [17]:
complete_vessel_df.to_csv('../data/processed_data/vessels.csv')
train_df.to_csv('../data/processed_data/train.csv', index=False)
test_df.to_csv("../data/processed_data/test.csv", index=False)

In [18]:
print(len(train_df))
print(len(test_df))

1522065
51739
